## Load Environments

In [1]:
%matplotlib inline

from __future__ import print_function
from __future__ import division
import warnings; warnings.filterwarnings('ignore')

import time
import random
from math import log

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

random.seed(125)
np.random.seed(137)

In [88]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV, LeavePGroupsOut, LeaveOneGroupOut, ParameterGrid
from sklearn.manifold import TSNE
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [61]:
from numerapi.numerapi import NumerAPI
from sklearn.externals import joblib

## Read in raw data sets, training, tournament (val, test, live)

In [3]:
!ls /input

numerai_tournament_data.csv  numerai_training_data.csv


In [4]:
def import_data_sets():
    train = pd.read_csv('/input/numerai_training_data.csv', index_col=0).drop('data_type', axis=1)
    df = pd.read_csv('/input/numerai_tournament_data.csv', index_col=0)
    valid = df.loc[df['data_type']=='validation'].drop('data_type', axis=1)
    test = df.loc[df['data_type']=='test'].drop('data_type', axis=1)
    live = df.loc[df['data_type']=='live'].drop('data_type', axis=1)
    return(train, valid, test, live)

In [5]:
train, valid, test, live = import_data_sets()

In [6]:
feature_cols = [f for f in train.columns if "feature" in f]

In [7]:
x_train = train[feature_cols]
x_val = valid[feature_cols]
x_test = test[feature_cols]
x_live = live[feature_cols]
y_train = train['target']
y_val = valid['target']


train_eras = train['era'].values
val_eras = valid['era'].values

In [20]:
def calc_consistency(labels, preds, eras):
    """ Calculate the consistency score.

    Args:
        labels: (np array) The correct class ids
        preds:  (np array) The predicted probabilities for class 1
        eras:   (np array) The era each sample belongs to
    """
    unique_eras = np.unique(eras)
    better_than_random_era_count = 0
    for era in unique_eras:
        this_era_filter = [eras == era]
        logloss = log_loss(labels[this_era_filter], preds[this_era_filter])
        
        print(logloss)
        
        if logloss < -np.log(0.5):
            better_than_random_era_count += 1

    consistency = better_than_random_era_count / float(len(unique_eras)) * 100
    print(consistency)

## Grid Search CV Pipeline - Logistic Regression

In [57]:
pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('reduce_dim', PCA()),
    ('classify', LogisticRegression())
])

N_DIM = [10, 16, 25]
C_OPTIONS = [0.01, 0.001]
PENALTIES = ['l1', 'l2']

param_grid = [{
        'reduce_dim__n_components': N_DIM,
        'classify__penalty': PENALTIES,
        'classify__C': C_OPTIONS
    }]

lr_grid = GridSearchCV(pipe, cv=5, n_jobs=1,
                       param_grid=param_grid, verbose=2, scoring='neg_log_loss')

In [58]:
lr_grid.fit(x_train.values, y_train.values, groups=train_eras)

lr_model = lr_grid.best_estimator_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=10 
[CV]  classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=10, total=   3.0s
[CV] classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.4s remaining:    0.0s


[CV]  classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=10, total=   3.0s
[CV] classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=10 
[CV]  classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=10, total=   2.9s
[CV] classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=10 
[CV]  classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=10, total=   2.9s
[CV] classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=10 
[CV]  classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=10, total=   2.9s
[CV] classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=16 
[CV]  classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=16, total=   3.6s
[CV] classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=16 
[CV]  classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=16, total=   3.6s
[CV] classify__C=0.01, classify__penalty=l1, reduce_dim__n_components=16 
[CV]  classify__C=0.01

[CV]  classify__C=0.001, classify__penalty=l2, reduce_dim__n_components=16, total=   4.4s
[CV] classify__C=0.001, classify__penalty=l2, reduce_dim__n_components=16 
[CV]  classify__C=0.001, classify__penalty=l2, reduce_dim__n_components=16, total=   4.4s
[CV] classify__C=0.001, classify__penalty=l2, reduce_dim__n_components=16 
[CV]  classify__C=0.001, classify__penalty=l2, reduce_dim__n_components=16, total=   4.4s
[CV] classify__C=0.001, classify__penalty=l2, reduce_dim__n_components=16 
[CV]  classify__C=0.001, classify__penalty=l2, reduce_dim__n_components=16, total=   4.4s
[CV] classify__C=0.001, classify__penalty=l2, reduce_dim__n_components=25 
[CV]  classify__C=0.001, classify__penalty=l2, reduce_dim__n_components=25, total=   5.3s
[CV] classify__C=0.001, classify__penalty=l2, reduce_dim__n_components=25 
[CV]  classify__C=0.001, classify__penalty=l2, reduce_dim__n_components=25, total=   5.3s
[CV] classify__C=0.001, classify__penalty=l2, reduce_dim__n_components=25 
[CV]  clas

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  4.3min finished


In [59]:
print(lr_grid.best_params_)
print(lr_grid.best_score_)

{'classify__C': 0.001, 'classify__penalty': 'l2', 'reduce_dim__n_components': 16}
-0.692758620326


In [102]:
#joblib.dump(lr_model, 'lr_model.pkl')
lr_model = joblib.load('lr_model.pkl') 

In [62]:
calc_consistency(y_val.values, lr_model.predict_proba(x_val.values)[:,1], val_eras)

0.692708284818
0.691789735792
0.692840650871
0.691724252343
0.692949190806
0.69188486236
0.693575069541
0.692723083929
0.69078442334
0.692858707377
0.69262144454
0.693991212471
83.33333333333334


## GridSearch CV Pipeline - Polynomial Features LR Classifier

In [98]:
pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('reduce_dim', PCA(n_components=16)),
    ('polyf', PolynomialFeatures(interaction_only=True)),
    ('classify', LogisticRegression(C=0.01, penalty='l2'))
])

DEGREES = [2,3]

param_grid = [{
        'polyf__degree': DEGREES
    }]

lr_grid2 = GridSearchCV(pipe, cv=5, n_jobs=1,
                       param_grid=param_grid, verbose=2, scoring='neg_log_loss')

In [99]:
lr_grid2.fit(x_train.values, y_train.values, groups=train_eras)

lr_model2 = lr_grid2.best_estimator_

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] polyf__degree=2 .................................................
[CV] .................................. polyf__degree=2, total=  15.1s
[CV] polyf__degree=2 .................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.8s remaining:    0.0s


[CV] .................................. polyf__degree=2, total=  15.9s
[CV] polyf__degree=2 .................................................
[CV] .................................. polyf__degree=2, total=  17.9s
[CV] polyf__degree=2 .................................................
[CV] .................................. polyf__degree=2, total=  15.1s
[CV] polyf__degree=2 .................................................
[CV] .................................. polyf__degree=2, total=  18.0s
[CV] polyf__degree=3 .................................................
[CV] .................................. polyf__degree=3, total= 1.1min
[CV] polyf__degree=3 .................................................
[CV] .................................. polyf__degree=3, total=  59.0s
[CV] polyf__degree=3 .................................................
[CV] .................................. polyf__degree=3, total= 1.2min
[CV] polyf__degree=3 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  7.9min finished


In [100]:
print(lr_grid2.best_params_)
print(lr_grid2.best_score_)

{'polyf__degree': 3}
-0.692832785214


In [101]:
joblib.dump(lr_model2, 'lr_model2.pkl')
#lr_model2 = joblib.load('lr_model2.pkl') 

['lr_model2.pkl']

In [103]:
calc_consistency(y_val.values, lr_model2.predict_proba(x_val.values)[:,1], val_eras)

0.692979687973
0.691487917565
0.693370222438
0.690726041919
0.693452892225
0.691585715501
0.694007185705
0.692776445011
0.689948727747
0.692461720562
0.692450878097
0.694499066454
66.66666666666666


## GridSearch CV Pipeline - XGBoost Classifier

In [23]:
pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('classify', XGBClassifier(eval_metric='logloss', n_jobs=-1, silent=False))
])

N_EST = [60]
LR = [0.01]
DEPTH = [10]

param_grid = [{
        'classify__n_estimators': N_EST,
        'classify__learning_rate': LR,
        'classify__max_depth': DEPTH        
    }]

xgb_grid = GridSearchCV(pipe, cv=LeavePGroupsOut(n_groups=40), n_jobs=1,
                        param_grid=param_grid, verbose=2, scoring='neg_log_loss')

In [24]:
xgb_grid.fit(x_train.values, y_train.values, groups=train_eras)

xgb_model = xgb_grid.best_estimator_

Fitting 120 folds for each of 1 candidates, totalling 120 fits
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  29.5s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   30.4s remaining:    0.0s


[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  29.6s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  29.9s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  29.9s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  29.5s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  29.0s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify

[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  29.4s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  29.9s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  29.2s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  30.0s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  30.1s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify

[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  28.8s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  29.1s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  29.1s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  29.3s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01, total=  28.9s
[CV] classify__n_estimators=60, classify__max_depth=10, classify__learning_rate=0.01 
[CV]  classify__n_estimators=60, classify__max_depth=10, classify

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 60.2min finished


In [25]:
print(xgb_grid.best_params_)
print(xgb_grid.best_score_)

{'classify__n_estimators': 60, 'classify__max_depth': 10, 'classify__learning_rate': 0.01}
-0.6929548569


In [26]:
joblib.dump(xgb_model, 'xgb_model.pkl')
xgb_model = joblib.load('xgb_model.pkl') 

In [28]:
calc_consistency(y_val.values, xgb_model.predict_proba(x_val.values)[:,1], val_eras)

0.693316586804
0.692225650055
0.693679347491
0.69227662318
0.69289980276
0.692962287434
0.693271446572
0.69335960866
0.691672713467
0.692532670685
0.692913702392
0.693839281121
58.333333333333336


## Random Forest Model

In [63]:
pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('reduce_dim', PCA()),
    ('classify', RandomForestClassifier(n_jobs=-1, verbose=2, random_state=73))
])

N_DIM = [10, 16, 25]
N_EST = [25, 50, 80]
MAX_DEPTH = [5, 10, 20]

param_grid = [{
        'reduce_dim__n_components': N_DIM,
        'classify__n_estimators': N_EST,
        'classify__max_depth': MAX_DEPTH
    }]

rf_grid = GridSearchCV(pipe, cv=5, n_jobs=1,
                        param_grid=param_grid, verbose=2, scoring='neg_log_loss')

In [64]:
rf_grid.fit(x_train.values, y_train.values, groups=train_eras)

rf_model = rf_grid.best_estimator_

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] classify__n_estimators=25, classify__max_depth=5, reduce_dim__n_components=10 
building tree 1 of 25building tree 2 of 25building tree 3 of 25building tree 4 of 25building tree 5 of 25building tree 7 of 25building tree 8 of 25






building tree 6 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25building tree 14 of 25

building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:    2.6s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    3.0s finished
[Parallel(n_jobs=8)]: Done  23 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Done  23 out of  25 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.3s remaining:    0.0s


[CV]  classify__n_estimators=25, classify__max_depth=5, reduce_dim__n_components=10, total=   5.9s
[CV] classify__n_estimators=25, classify__max_depth=5, reduce_dim__n_components=10 
building tree 1 of 25building tree 2 of 25building tree 3 of 25building tree 4 of 25building tree 5 of 25building tree 7 of 25building tree 6 of 25






building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:    2.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    3.0s finished
[Parallel(n_jobs=8)]: Done  23 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Done  23 out of  25 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.1s finished


[CV]  classify__n_estimators=25, classify__max_depth=5, reduce_dim__n_components=10, total=   5.5s
[CV] classify__n_estimators=25, classify__max_depth=5, reduce_dim__n_components=10 
building tree 1 of 25building tree 2 of 25building tree 3 of 25building tree 4 of 25building tree 5 of 25building tree 6 of 25building tree 7 of 25






building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25building tree 15 of 25

building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


...

[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed: 39.6min finished


building tree 1 of 50building tree 2 of 50building tree 3 of 50building tree 4 of 50building tree 5 of 50building tree 8 of 50





building tree 6 of 50
building tree 7 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.2s


building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50building tree 47 of 50

building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    7.1s finished


In [65]:
print(rf_grid.best_params_)
print(rf_grid.best_score_)

{'classify__n_estimators': 50, 'classify__max_depth': 5, 'reduce_dim__n_components': 10}
-0.692823565237


In [66]:
joblib.dump(rf_model, 'rf_model.pkl')
#rf_model = joblib.load('rf_model.pkl') 

['rf_model.pkl']

In [67]:
calc_consistency(y_val.values, rf_model.predict_proba(x_val.values)[:,1], val_eras)

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished


0.69217419868
0.69136241285
0.692977870444
0.692662864181
0.69208463949
0.692729669791
0.693354743035
0.692913053723
0.691437053556
0.692642917427
0.692786441803
0.693989096434
83.33333333333334


## Ensemble

In [104]:
to_pred = pd.concat([x_val, x_test, x_live])

xgb_preds = pd.Series(xgb_model.predict_proba(to_pred.values)[:,1])
lr_preds = pd.Series(lr_model.predict_proba(to_pred.values)[:,1])
rf_preds = pd.Series(rf_model.predict_proba(to_pred.values)[:,1])
lr_preds2 = pd.Series(lr_model2.predict_proba(to_pred.values)[:,1])

to_vote = pd.concat([xgb_preds, lr_preds, lr_preds2, rf_preds], axis=1)

# Ensemble Tree
ens_preds = to_vote.mean(axis=1)
#ens_preds = lr_preds

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.1s finished


In [105]:
ens_preds.describe()

count    243470.000000
mean          0.500017
std           0.012839
min           0.426344
25%           0.491641
50%           0.500790
75%           0.509098
max           0.550663
dtype: float64

In [106]:
sub = pd.concat([pd.Series(to_pred.index), pd.Series(ens_preds)], axis=1)
sub.columns = ['id', 'probability']
sub.head()

,id,probability
0,n61d97b708905409,0.501159
1,n350462f2ea6f467,0.494400
2,n9451f9eda57c499,0.479408
3,n3eeda8f0c131475,0.512462
4,nad2c6fb811fb4bb,0.489441


In [110]:
pred_path = 'sub_sk_88_' + str(int(time.time())) + '.csv'

sub.to_csv(pred_path, index=False)

In [108]:
pred_path

'sub_sk_88_1514680857.csv'

In [80]:
def get_model_key(model):
    
    if(model == "spacekitty"):
        return('6GHLWWOGPWHBN5ODRPJFCXVV6U4C5WBB','HOZSYDP45H7FCVV3N2QWRZVXTRYWS74QODA2ON5TOUHRJLG3STJOU63M2ZVYTZGN')


In [73]:
def upload_preds(path_to_preds, pub, priv):
    # set example username and round
    example_public_id = pub
    example_secret_key = priv

    # some API calls do not require logging in
    napi = NumerAPI(verbosity="info")
    
    # download current dataset
    napi.download_current_dataset(unzip=True)
    
    # provide api tokens
    napi = NumerAPI(example_public_id, example_secret_key)

    # upload predictions
    submission_id = napi.upload_predictions(path_to_preds)

In [109]:
pub, priv = get_model_key('spacekitty')

upload_preds(pred_path, pub, priv)

2017-12-31 00:41:02,613 INFO numerapi.numerapi: downloading current dataset...
2017-12-31 00:41:02,615 INFO numerapi.numerapi: target file already exists
2017-12-31 00:41:02,615 INFO numerapi.numerapi: uploading prediction...
2017-12-31 00:41:02,617 INFO requests.packages.urllib3.connectionpool: Starting new HTTPS connection (1): api-tournament.numer.ai
2017-12-31 00:41:02,862 INFO requests.packages.urllib3.connectionpool: Starting new HTTPS connection (1): numerai-production-uploads.s3.amazonaws.com
2017-12-31 00:41:03,481 INFO requests.packages.urllib3.connectionpool: Starting new HTTPS connection (1): api-tournament.numer.ai
